# Using Ubuntu & cmd center  [WSL]

## Terminal Ubuntu keyboard code

1. add

Directory from terminal
** # explorer.exe . ** 

### Your terminal’s way of saying:

> **“You’re user `bhuns`, logged into a machine called `Puget1`, currently sitting in your home directory (`~`).”**

### What Each Part Means

| Terminal Segment       | Meaning                                                                 |
|------------------------|-------------------------------------------------------------------------|
| `bhuns`                | Your **Linux username** inside WSL (Windows Subsystem for Linux)        |
| `@Puget1`              | The **hostname** of your machine (likely your Windows PC’s name)        |
| `:~`                   | You're in your home directory (`~` = `/home/bhuns`)                     |
| `$`                    | The **shell prompt** symbol for a regular (non-root) user               |

---

### 🧠 Why It Says That

- WSL sets your **user** and **computer name** into the prompt, similar to Linux and macOS terminals.
- The terminal prompt is defined by an environment variable called `$PS1`.

For example, in your case it’s probably set to something like:

Which expands to:
- `\u` → current user: `bhuns`
- `\h` → hostname: `Puget1`
- `\w` → current working directory
- `\$` → prompt character (`$` or `#` if you're root)

### 🎨 Want to Customize It?

You can personalize the prompt to include Git branch info, colors, or even emoji. Just edit your `~/.bashrc`:

```bash
nano ~/.bashrc
```

Look for the line that sets `PS1` and tweak away—or I can whip up a smart version that even shows your active Conda environment and Git branch. Want that kind of tricked-out prompt?

# WSL cheat sheet


**WSL (Linux)** and **Windows** filesystems are connected &
How to navigate between them easily:
---

### 🧭 WSL ↔ Windows File System Navigation
## open configuration files: **nano ~/.bashrc**

| **Action** | **In WSL (Ubuntu Bash)** | **In Windows (Explorer)** | **What It Does** |
|------------|---------------------------|----------------------------|------------------|
| Access **C:\Users** | `/mnt/c/Users/` | `C:\Users\` | Lets WSL read/write to your Windows files |
| Go to WSL **Home (`~`)** | `cd ~` or `cd /home/bhuns` | `\\wsl$\Ubuntu\home\bhuns` | Access your WSL/Linux files from Windows |
| Open current folder in Explorer | `explorer.exe .` | N/A | Launches Windows Explorer in current WSL dir |
| Launch Notepad from WSL | `notepad.exe filename.txt` | N/A | Opens a file using Windows Notepad |
| Map WSL folder as drive | (N/A) | Use `Map Network Drive` to `\\wsl$\Ubuntu\...` | Mount Linux files as a Windows drive |

> 🧠 Replace `Ubuntu` with your actual distro name if needed, like `Ubuntu-22.04`

---

### 🗂️ Folder Mapping Examples

| **Linux Path**               | **Windows View**                          |
|-----------------------------|-------------------------------------------|
| `/home/bhuns/JL_1`          | `\\wsl$\Ubuntu\home\bhuns\JL_1`          |
| `/mnt/c/Users/bhuns/Desktop`| `C:\Users\bhuns\Desktop`                  |
| `/mnt/d/Projects`           | `D:\Projects`                             |

---

Would you like a printable PDF or PNG version of this for your desktop or workspace? I can draw it up in seconds as a quick-reference chart.

## Where to launch **jupyter labs**  (using WSL) [frm copilot]

### 🗂️ Where to Run Your Commands

Here’s the rundown of where each type of command should live:

| **Command**                              | **Run From**              | **Why**                                                                 |
|------------------------------------------|----------------------------|--------------------------------------------------------------------------|
| `conda activate scu`                     | Anywhere                   | Activates your environment globally; doesn’t depend on your folder.     |
| `jupyter lab`                            | **From your project dir** (e.g. `~/scu-project`) | So JupyterLab opens with your notebooks right there.                   |
| `git add`, `commit`, `push`, etc.        | **From inside your repo** (e.g. `~/scu-project`) | Git actions only apply to the repo you're inside.                      |

---

### ✅ Your Ideal Workflow (All Together)

```bash
cd ~/scu-project         # 1. Go to your project folder
conda activate scu       # 2. Activate your Conda environment
jupyter lab              # 3. Start JupyterLab from inside the project
```

> This ensures your notebooks open from the correct location, your Git status reflects the right repo, and your environment is loaded properly.

# Updated Strategy for Preserving Formulas [this is Def see .py versions done later]




We’ll upgrade your workflow like this:

### 1. **Load Excel with Formulas Intact**  
Instead of relying on `pandas.read_excel`, we’ll use `openpyxl` to:
- Read cell values **and formulas**
- Build a DataFrame that holds formulas where applicable

### 2. **Write Back While Keeping Formulas**  
When writing, we’ll:
- Loop through the DataFrame
- Preserve formulas by re-inserting them into the Excel cells, rather than overwriting with static values

---

## 📦 Enhanced `load_df` Function (Preserve Formulas)

```python
from openpyxl import load_workbook
import pandas as pd

def load_df_with_formulas(file_name="xl_tbl.xlsx", sheet_name="xl_tbl"):
    wb = load_workbook(file_name, data_only=False)  # `data_only=False` preserves formulas
    ws = wb[sheet_name]

    data = []
    for row in ws.iter_rows(values_only=False):
        row_data = []
        for cell in row:
            if cell.value is not None:
                if cell.data_type == 'f':  # It's a formula
                    row_data.append(f"={cell.value}")
                else:
                    row_data.append(cell.value)
            else:
                row_data.append(None)
        data.append(row_data)

    # Extract headers from first row
    headers = data[0]
    df = pd.DataFrame(data[1:], columns=headers)
    print(f"✅ Loaded sheet '{sheet_name}' from '{file_name}', preserving formulas.")
    return df
```

---

## 📤 Enhanced `write_df` Function (Write Back Formulas)

```python
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

def write_df_with_formulas(df, file_name="xl_tbl.xlsx", sheet_name="xl_tbl", table_name="xl_tbl"):
    wb = Workbook()
    ws = wb.active
    ws.title = sheet_name

    # Write headers
    for col_idx, col_name in enumerate(df.columns, start=1):
        ws.cell(row=1, column=col_idx, value=col_name)

    # Write values or formulas
    for row_idx, row in enumerate(df.itertuples(index=False), start=2):
        for col_idx, value in enumerate(row, start=1):
            cell = ws.cell(row=row_idx, column=col_idx)
            if isinstance(value, str) and value.startswith("="):
                cell.value = value[1:]  # Remove leading '=' for Excel formula
                cell.data_type = 'f'
            else:
                cell.value = value

    # Define table
    nrows, ncols = df.shape
    table_range = f"A1:{get_column_letter(ncols)}{nrows+1}"

    from openpyxl.worksheet.table import Table, TableStyleInfo
    table = Table(displayName=table_name, ref=table_range)
    style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True)
    table.tableStyleInfo = style
    ws.add_table(table)

    wb.save(file_name)
    print(f"💾 Wrote updated DataFrame to Excel file '{file_name}', preserving formulas.")
```

---

**These enhanced functions will respect your formula logic and keep the round-trip editing seamless. Would you like me to wrap both into a module like `xl_formula_sync.py` with built-in checks for formula integrity? You could use it across any number of `_tbl` projects without reinventing the wheel.**

# Git handling renamed and deleted files

### 🗂️ How Git Handles Renamed Files
Git doesn’t track renames explicitly. Instead, it uses **content similarity** to detect renames during `git diff` or `git log`:
- If you rename a file manually (e.g., `mv old.py z_old.py`), Git sees it as:
  - One file deleted (`old.py`)
  - One file added (`z_old.py`)
- During a diff or log, Git may infer it was a rename if the content is **similar enough** (default threshold is ~50%).

> ✅ If you want Git to **explicitly track the rename**, use `git mv old.py z_old.py`. This stages the change as a rename and preserves history more cleanly.

---

### 🧹 How Git Handles Deleted Files
- If you delete a file manually, Git sees it as a deletion.
- You can stage the deletion with:
  ```bash
  git add -u
  ```
  or
  ```bash
  git rm filename
  ```
- Once committed, the file is removed from the repo, but its history remains accessible.

> You can always restore a deleted file using:
  ```bash
  git checkout HEAD^ -- path/to/file
  ```

---

### 🧠 Should You Change Your Procedure?
Your `z_` prefix method is fine for **local organization**, but here’s how to make it Git-friendly:
- Use `git mv` instead of manual renaming to preserve history.
- When ready to delete, use `git rm` or `git add -u` to stage the deletion.
- Optionally, automate cleanup with a script that finds `z_*` files older than X days and removes them with Git commands.

Would you like help writing a cleanup script that stages and commits deletions for all `z_*` files older than a certain threshold? That could streamline your workflow beautifully.

# Push and commit help from Copilot

### 🌱 Local vs. Remote Branches

| Branch Label       | What It Represents                                  |
|--------------------|-----------------------------------------------------|
| `main`             | Your **local** main branch                         |
| `sandbox`          | Your **local** sandbox branch                      |
| `origin/main`      | The **main branch** on the **remote repo** (`origin`) |
| `origin/sandbox`   | The **sandbox branch** on the **remote repo** (`origin`) |

So:

- The ones without `origin/` are the branches you’re working with locally.
- The `origin/*` branches reflect what’s currently on the remote server (like GitHub).

---

### 🔄 How They Interact

- When you **push**, your local branch updates the corresponding `origin/*` branch.
- When you **pull**, your local branch fetches changes from its `origin/*` counterpart.
- If they differ, you’ll see divergence in commit counts or changes not yet synced.

---

### 🔍 Example Workflow

If you make changes on `sandbox` locally and push them:

```bash
git push origin sandbox
```

Then `origin/sandbox` updates to match your local `sandbox`.

---


## Creating .py versions of def functions

# Storing variables in a "shared_data.py" and other options

## What Happens When You Import a Python Module in a Notebook

### 🧠 What Happens When You Import a Python Module in a Notebook

- ✅ **Functions are loaded into memory**: When you import your module (e.g. `import F_my_utils`), Python reads the file and makes its functions and variables available in the current notebook's namespace.
- 📌 **Scoped to the current notebook/kernel**: The imported functions are only available in the notebook where you ran the import—**not across all notebooks** unless they share the same kernel session.

---

### 🧪 Example Scenario

If you open `Notebook A` and run:

```python
import F_my_utils
F_my_utils.backup_excel("data/file.xlsx")
```

It works beautifully. But if you then open `Notebook B` in a separate tab and don’t rerun `import F_my_utils` there—it won’t recognize the functions because that notebook has its own kernel and environment.

---

### 🧩 Shared Activation? Only via Shared Kernel

Technically, if multiple notebooks are connected to the **same kernel**, the import could carry over. But in most cases—especially in modular workflows like yours—each notebook has an isolated kernel session.

---

### 🔧 What You Can Do

- Put shared functions in a custom utility module (`F_my_utils.py`)
- Import that module at the top of each notebook where needed
- You can even automate it with a launcher cell in your sandbox dashboard that runs on startup

---

Want help creating a startup cell that confirms the module is loaded and shows which functions are available? That could make your notebooks feel unified, even across sessions.

In [1]:
## options

### Options for data storage


### 🥒 Option 1: Use `pickle` for serialization
```python
import pickle

# Save the DataFrame
with open('df_tbl.pkl', 'wb') as f:
    pickle.dump(df_tbl, f)

# In another notebook
with open('df_tbl.pkl', 'rb') as f:
    df_tbl = pickle.load(f)
```
- ✅ Fast, native to Python, simple
- ⚠️ Not human-readable, and version compatibility might be an issue across Python or pandas updates

---

### 📄 Option 2: Export to CSV or Excel
If the table came from Excel, you can "round-trip" it by saving a version that preserves structure or formulas if needed:
```python
df_tbl.to_csv('df_tbl.csv', index=False)
# or
df_tbl.to_excel('df_tbl.xlsx', index=False)
```
- ✅ Human-readable and portable
- ⚠️ Formulas aren’t preserved unless you're using tools like `openpyxl` to manage that explicitly

---

### 🧠 Option 3: Use a shared Python module
If you're trying to use `df_tbl` across notebooks without re-importing or loading it each time:
1. Create a Python file (e.g. `shared_data.py`)
2. Save `df_tbl` creation code in it
3. Import it from other notebooks:
```python
from shared_data import df_tbl
```
- ✅ Keeps your logic centralized and versioned via Git
- ⚠️ Requires re-execution, not storage-based reuse

---

### 🧰 Option 4: Joblib (alternative to pickle)
```python
import joblib
joblib.dump(df_tbl, 'df_tbl.joblib')
df_tbl = joblib.load('df_tbl.joblib')
```
- ✅ Better for larger data, supports compression
- ⚠️ Still binary format, not human-readable


### 📁 Step 1: Create a Python module to store the shared DataFrame

Let’s say your original notebook imports `df_tbl` from an Excel file using `xl_tbl`.

#### **shared_data.py**
```python
import pandas as pd

# Define the path to the Excel file
excel_path = 'data/xl_tbl.xlsx'

# Load the table into a DataFrame
df_tbl = pd.read_excel(excel_path, sheet_name='Sheet1')
```

Save this as `shared_data.py` in the same directory or in a known module path.

---

### 📓 Step 2: Access the DataFrame in another notebook

You can now import the `df_tbl` directly in any notebook:

```python
from shared_data import df_tbl

# Use df_tbl just like any other DataFrame
print(df_tbl.head())
```

---

### 💡 Tips for this pattern

- If your Excel file changes frequently and you want `df_tbl` to reflect those changes dynamically, you could wrap it in a function:
```python
# In shared_data.py
def load_table():
    return pd.read_excel('data/xl_tbl.xlsx', sheet_name='Sheet1')
```

Then in your notebook:
```python
from shared_data import load_table
df_tbl = load_table()
```

- This setup plays very nicely with Git—your shared module is version-controlled and changes are traceable.
- You can extend this module to include multiple shared tables or utility functions.

---

Want help turning this into a reusable project scaffold or organizing it inside Pages? You've got quite the system-thinker mind—I think you'd crush it with a template setup.

# Tags

Tags in Jupyter notebooks are like invisible labels you attach to individual cells—they live in the cell’s metadata and can be used to control behavior, organize content, or trigger automation. Think of them as behind-the-scenes flags that tools like `nbconvert`, `papermill`, or even your own scripts can respond to.

---

### 🏷️ What Tags Can Do

- **Hide cells or inputs** during export (e.g. `remove-input`, `hide-cell`)
- **Mark parameter cells** for automation tools like `papermill`
- **Skip execution** or expect exceptions (e.g. `raises-exception`)
- **Organize content** for indexing or filtering
- **Trigger custom behavior** in scripts or dashboards

---

### ✍️ How to Add Tags in JupyterLab

1. Open your notebook in JupyterLab.
2. Click the **gear icon** (⚙️) in the top-right corner to open the **Property Inspector**.
3. Select a cell and scroll to the **Tags** section.
4. Type a tag name (e.g. `Parameters`, `hide-input`) and press Enter.

You can add multiple tags to a single cell.

---

### 🧪 Example Use Case

Let’s say you want to hide a setup cell when exporting to HTML:

```python
# This cell sets up paths and imports
import pandas as pd
import os
```

Add the tag: `remove-input`

When you run `nbconvert` to export the notebook, the input code will be hidden, but the output (if any) remains visible.

---

### 🧠 Bonus: Use Tags for Search & Indexing

You can create your own tags like `#tags: mito, NAD, backup` in markdown cells. Then search for `#tags:` to quickly locate relevant sections—even if you don’t remember the full word.

---

Would you like help building a tag-based index or search dashboard for your notebooks? It could make navigating your workflow much smoother.

# File Filter(in the File Browser Sidebar) & Ctrl+F for searching

Great question, Bill—these two search tools in JupyterLab serve very different purposes, even though they both help you find things.

---

### 🗂️ File Filter (in the File Browser Sidebar)

- **Scope**: Searches filenames and folder names only.
- **Use Case**: Quickly locate a file like `xl_xf_xl_py.ipynb` or a folder like `backups/`.
- **Matching**: Uses **substring matching**—so typing `xl_` will match `xl_xf_xl_py.ipynb`.
- **Limitations**: Doesn’t search inside file contents or metadata.

---

### 🔍 Ctrl+F (Notebook or Editor Search)

- **Scope**: Searches **inside** the open file—code cells, markdown, outputs.
- **Use Case**: Find a variable name, comment, or keyword inside a notebook or text file.
- **Matching**:
  - Supports **regex** (click the `.*` icon).
  - Can search **cell outputs** (though jumping to results may break if output is collapsed).
  - Doesn’t search across multiple files.

---

### 🧠 Summary

| Feature            | File Filter Sidebar        | Ctrl+F in Notebook       |
|--------------------|----------------------------|--------------------------|
| Searches filenames | ✅                          | ❌                        |
| Searches contents  | ❌                          | ✅                        |
| Supports regex     | ❌                          | ✅                        |
| Scope              | All files/folders visible  | Current open file only   |

---

If you're trying to find a specific notebook but can't remember the full name, use the file filter. If you're inside a notebook and want to locate a function or tag, use Ctrl+F. Want help building a fuzzy search tool that spans both filenames and notebook contents? We could wrap it into your dashboard.

# Personal Workflow Launcher (Starter Notebook Header)

```python
# 🔍 Environment Diagnostic
import importlib.util, subprocess

def check_ready(pkgs):
    for pkg in pkgs:
        if importlib.util.find_spec(pkg) is None:
            print(f"⚠️ Missing: {pkg}")
            subprocess.run(["pip", "install", pkg])
        else:
            print(f"✅ Found: {pkg}")

check_ready(["ipywidgets", "IPython", "nbdime", "F_my_utils"])

# 🧬 Git Status Snapshot
def git_snapshot(file="xl_xf_xl_py.ipynb"):
    branch = subprocess.check_output(["git", "rev-parse", "--abbrev-ref", "HEAD"], text=True).strip()
    commit = subprocess.check_output(["git", "log", "-1", "--pretty=format:%h – %s"], text=True).strip()
    status = subprocess.check_output(["git", "status", "--short", file], text=True).strip()
    print(f"\n📁 Git Branch: {branch}\n📝 Last Commit: {commit}\n📌 File Status: {'Modified' if status else 'Clean'}")

git_snapshot()
```

---

## 🧰 Suggested Dashboard Sections

### ✅ 1. **Notebook Launcher**
Button or dropdown to open your most-used `.ipynb` files

### 🔄 2. **Git Actions**
- Stash and switch branches
- Compare commits (launch `nbdiff-web`)
- Auto-tag versions

### 📦 3. **Module & Data Check**
Show which utility modules and data sets are loaded

### 🧠 4. **Markdown Index**
Outline links to Copilot Pages and local projects (add `#tags` for fast search)

---

Would you like me to build this into a runnable notebook or convert it into a Copilot Page template for wider use? You’re not far from turning this into a personal control panel for code, data, and experiments.